In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model

In [2]:
tf.__version__

'2.2.0'

In [3]:
import matplotlib.pyplot as plt
import string
import re

In [4]:
data=pd.read_csv('HE-Data.csv')

In [5]:
data.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,indic2012,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,indic2012,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,tides,The then Governor of Kashmir resisted transfer...,कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का व...
6,indic2012,In this lies the circumstances of people befor...,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,indic2012,“”Global Warming“” refer to warming caused in ...,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई...
9,tides,You may want your child to go to a school that...,हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटे...


In [6]:
data=data[data['source']=='ted']
data.head(10)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,to turn on the lights or to bring him a glass ...,"लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [7]:
data.isnull().sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [8]:
data.drop_duplicates(inplace=True)

In [9]:
len(data)

38803

In [10]:
data['english_sentence'] = data['english_sentence'].apply(lambda x:x.lower())
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x:x.lower())

punc=set(string.punctuation)
data['english_sentence'] = data['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in punc))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in punc))

In [11]:
dig=str.maketrans('','', string.digits)
data['english_sentence']=data['english_sentence'].apply(lambda x: x.translate(dig))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.translate(dig))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub('[२३०८१५७९४६]','', x))

data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.strip())
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.strip())

In [12]:
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("'",'', x))
data['english_sentence'] = data['english_sentence'].apply(lambda x: re.sub("'",'', x))



In [13]:
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x:'START_ ' + x +' _END')
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...
1,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...
3,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...
7,ted,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...
13,ted,so there is some sort of justice,START_ तो वहाँ न्याय है _END


In [14]:
data['eng_length'] = data['english_sentence'].apply(lambda x: len(x.split()))
data['hin_length'] = data['hindi_sentence'].apply(lambda x: len(x.split()))
data.head(10)

,source,english_sentence,hindi_sentence,eng_length,hin_length
0,ted,politicians do not have permission to do what ...,START_ राजनीतिज्ञों के पास जो कार्य करना चाहिए...,12,15
1,ted,id like to tell you about one such child,START_ मई आपको ऐसे ही एक बच्चे के बारे में बता...,9,13
3,ted,what we really mean is that theyre bad at not ...,START_ हम ये नहीं कहना चाहते कि वो ध्यान नहीं ...,12,13
7,ted,and who are we to say even that they are wrong,START_ और हम होते कौन हैं यह कहने भी वाले कि व...,11,15
13,ted,so there is some sort of justice,START_ तो वहाँ न्याय है _END,7,6
23,ted,this changed slowly,START_ धीरे धीरे ये सब बदला _END,3,7
26,ted,were being produced,START_ उत्पन्न नहीं कि जाती थी _END,3,7
30,ted,and you can see this led is going to glow,START_ और जैसा आप देख रहे है ये एलईडी जल उठेगी...,10,12
32,ted,to turn on the lights or to bring him a glass ...,START_ लाईट जलाने के लिए या उनके लिए पानी लाने...,13,13
35,ted,can you imagine saying that,START_ क्या आप ये कल्पना कर सकते है _END,5,9


In [15]:
eng_vocab=set()
hin_vocab=set()
for sentence in data['english_sentence']:
    for word in sentence.split():
        if word not in eng_vocab:
            eng_vocab.add(word)
            
for sentence in data['hindi_sentence']:
    for word in sentence.split():
        if word not in hin_vocab:
            hin_vocab.add(word)            
            
            
print(len(eng_vocab), len(hin_vocab))            

17345 22285


In [16]:
max_len_in=max(data['eng_length'])
max_len_out=max(data['hin_length'])
eng_voc=sorted(list(eng_vocab))
hin_voc=sorted(list(hin_vocab))
enc_tokens=len(eng_voc)
dec_tokens=len(hin_voc)

enc_tokens+=1
dec_tokens+=1
enc_tokens, dec_tokens

(17346, 22286)

In [17]:
from sklearn.utils import shuffle
data=shuffle(data)
data.head()

,source,english_sentence,hindi_sentence,eng_length,hin_length
55859,ted,♫ our babies come fullformed clonie ♫,START_ ♫हमारे बच्चे बड़े ही आते हैं कलोनी♫ _END,7,9
47923,ted,ok now what can you do reduce emissions in you...,START_ ठीक है अब आप क्या कर सकते हैं अपने घरो...,11,16
124920,ted,so we ran another group of babies,START_ तो हमने बच्चों के एक और समूह पर प्रयोग ...,7,12
112597,ted,and the forelimbs and the hind limbs of animals,START_ जानवरो के अगले पाँवों और पिछले पाँवो पर...,9,11
112072,ted,it represents the beginning,START_ यह पश्चिम को अफ़्रीका की कहानियाँ बताने...,4,10


In [18]:
input_token_index = dict([(word, i+1) for i, word in enumerate(eng_voc)])
target_token_index = dict([(word, i+1) for i, word in enumerate(hin_voc)])

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [19]:
from sklearn.model_selection import train_test_split

X,y = data['english_sentence'], data['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=4)

In [20]:
def train_gen(X=X_train, y=y_train, batch_size=128):
    while True:
        for i in range(0,len(X),batch_size):
        
            enc_input=np.zeros((batch_size, max_len_in), dtype='float32')
            dec_input=np.zeros((batch_size, max_len_out), dtype='float32')
            dec_output=np.zeros((batch_size, max_len_out, dec_tokens), dtype='float32')

            for j,(input_text, target_text) in enumerate(zip(X[i:i+batch_size], y[i:i+batch_size])):

                for t, word in enumerate(input_text.split()):
                    enc_input[j,t] = input_token_index[word]

                for t,word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        dec_input[j,t] = target_token_index[word]

                    if t>0:
                        dec_output[j,t-1,target_token_index[word]] = 1

            yield([enc_input, dec_input], dec_output)              

In [21]:
dim=300

In [22]:
enc_input = Input(shape=(None, ))
enc_emb = Embedding(enc_tokens, dim, mask_zero=True)(enc_input)
enc_lstm = LSTM(dim, return_state=True)
enc_output, state_h, state_c = enc_lstm(enc_emb)
enc_states =[state_h, state_c]

dec_input = Input(shape=(None, ))
dec_emb_layer = Embedding(dec_tokens, dim, mask_zero=True)
dec_emb = dec_emb_layer(dec_input)
dec_lstm = LSTM(dim, return_sequences=True, return_state=True)
dec_output, _, _ = dec_lstm(dec_emb, initial_state=enc_states)
dec_dense = Dense(dec_tokens, activation='softmax')
dec_out = dec_dense(dec_output)

model=Model([enc_input, dec_input], dec_out)





In [23]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    5203800     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    6685800     input_2[0][0]                    
______________________________________________________________________________________________

In [25]:
train_samples= len(X_train)
test_samples = len(X_test)
batch_size = 128




In [26]:
train_samples, test_samples

(31042, 7761)

In [ ]:
#for i in range(50):
#    model.fit_generator(generator=train_gen(X_train, y_train, batch_size=batch_size),
#                    steps_per_epoch=train_samples//batch_size, epochs=1,
#                     validation_data=train_gen(X_test, y_test, batch_size=batch_size),
#                     validation_steps=test_samples//batch_size)
with tf.device('/GPU:0'):
        model.fit(train_gen(X_train, y_train, batch_size=batch_size), epochs=50, steps_per_epoch=train_samples//batch_size, 
                  validation_data=train_gen(X_test, y_test, batch_size=batch_size), validation_steps=test_samples//batch_size)                    

Epoch 1/50


In [27]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [29]:
model.save_weights('nmt_weights.h5')

In [49]:
enc_model=Model(enc_input, enc_states)

dec_state_h = Input(shape=(dim, ))
dec_state_c = Input(shape=(dim, ))
dec_input_states = [dec_state_c, dec_state_h]
dec_emb2 = dec_emb_layer(dec_input)

dec_out2, state_h2, state_c2 = dec_lstm(dec_emb2, initial_state=dec_input_states)
dec_states2 = [state_h2, state_c2]
dec_output2 = dec_dense(dec_out2)

dec_model = Model ([dec_input] + dec_input_states, 
                   [dec_output2] + dec_states2)



In [50]:
def dec_sequence(input_seq):
    states = enc_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0,0]= target_token_index['START_']
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = dec_model.predict([target_seq] + states)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        
        states = [h, c]

    return decoded_sentence